In [1]:
import os

In [2]:
os.chdir("../../")

In [3]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DatasplitConfig:
    root_dir: Path
    feature_eng_data: Path
    X_train: Path
    X_test: Path
    y_train: Path
    y_test: Path
    test_size: float
    random_state: int

In [5]:
from src.bike.constants import *
from src.bike.utils.common import read_yaml,create_directory

[2024-09-29 22:21:26,891: INFO]: Logging file start


In [6]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directory([self.config.data_root])

    def get_data_split_config(self) -> DatasplitConfig:
        config = self.config.splliting_data
        params = self.params.split
        create_directory([config.root_dir])

        split_config = DatasplitConfig(
            root_dir = config.root_dir,
            feature_eng_data = config.feature_eng_data,
            X_train = config.X_train,
            X_test = config.X_test,
            y_train = config.y_train,
            y_test = config.y_test,
            test_size = params.test_size,
            random_state = params.random_state
        )

        return split_config

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [26]:
class SplitData:
    def __init__(self,config:DatasplitConfig):
        self.config = config
    
    def split_data(self):
        df = pd.read_csv(self.config.feature_eng_data)
        X = df.drop('price',axis=1)
        y = df['price']
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=self.config.test_size,random_state=self.config.random_state)
        print(f"X_train shape:{X_train.shape}")
        print(f"X_test shape:{X_test.shape}")
        print(f"y_train shape:{y_train.shape}")
        print(f"y_test shape:{y_test.shape}")
        print(X_train.head(2))
        return X_train,X_test,y_train,y_test
    
    def save_data(self):
        X_train,X_test,y_train,y_test = SplitData.split_data(self)
        X_train.to_csv(self.config.X_train,index = False)
        X_test.to_csv(self.config.X_test,index = False)
        y_train.to_csv(self.config.y_train,index = False)
        y_test.to_csv(self.config.y_test,index=False)
        print("data save complete")

    def remove_unwanted_cols(self):
        X_train = pd.read_csv(self.config.X_train)
        X_test = pd.read_csv(self.config.X_test)

        X_train.drop(columns='Unnamed: 0',axis=1,inplace=True)
        X_train.to_csv(self.config.X_train,index=False)
        X_test.drop(columns='Unnamed: 0',axis=1,inplace=True)
        X_test.to_csv(self.config.X_test,index=False)
        print("after clean:\n",X_train.head(1))
        print("after clean:\n",X_test.head(1))

In [25]:
try:
    config = ConfigurationManger()
    sp_config = config.get_data_split_config()
    split = SplitData(config=sp_config)
    split.split_data()
    split.save_data()
    split.remove_unwanted_cols()
except Exception as e:
    raise e

[2024-09-29 22:44:01,847: INFO]: yaml: config/bike_config.yaml loaded successfully
[2024-09-29 22:44:01,850: INFO]: yaml: params/bike_params.yaml loaded successfully
[2024-09-29 22:44:01,850: INFO]: created directory at: data/bike
[2024-09-29 22:44:01,851: INFO]: created directory at: data/bike/data_split
X_train shape:(26118, 8)
X_test shape:(6530, 8)
y_train shape:(26118,)
y_test shape:(6530,)
       Unnamed: 0  bike_name  city  kms_driven  owner   age    power  brand
31044       31044        151     3   -0.645916      0   6.0 -0.84441      5
24910       24910         56    10    0.434766      0  15.0 -0.47246      1
X_train shape:(26118, 8)
X_test shape:(6530, 8)
y_train shape:(26118,)
y_test shape:(6530,)
       Unnamed: 0  bike_name  city  kms_driven  owner   age    power  brand
31044       31044        151     3   -0.645916      0   6.0 -0.84441      5
24910       24910         56    10    0.434766      0  15.0 -0.47246      1
data save complete
after clean:
    bike_name  city  